<a href="https://colab.research.google.com/github/abhilasha-kumar/Connector/blob/master/search-models/search_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing packages and mounting drive

In [1]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import operator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import heapq
import pandas as pd
import scipy.spatial.distance
import numpy as np
from numpy.random import randint
from sklearn.preprocessing import MinMaxScaler, normalize
from numpy.linalg import matrix_power
from scipy import stats
from sklearn import preprocessing
from numpy.random import choice
import json
import heapq
import json
import itertools
import sys
import scipy.spatial.distance
from scipy.special import softmax



In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Importing embeddings

In [3]:
# import glove embeddings
representations = {}
parentfolder = "/content/drive/My Drive/Connector-CogSci2021/Descriptive analyses and model predictions/"
representations['glove'] = pd.read_csv(parentfolder +"/glove_embeddings.csv").transpose().values
vocab = pd.DataFrame(list(pd.read_csv(parentfolder +"/glove_embeddings.csv").columns), columns=["vocab_word"])
print(f"embeddings are shaped:", representations['glove'].shape)
print(f"vocab is {len(vocab)} words")

embeddings are shaped: (12218, 300)
vocab is 12218 words


# Defining search functions

In [216]:
def create_similarity_matrix(matrix, vector, N):
  matrix = 1-scipy.spatial.distance.cdist(matrix, vector, 'cosine').reshape(-1)
  matrix = matrix.reshape((N,N))
  return matrix

def create_graph(similarity_matrix, threshold):
  ## make diagonal 0 and all values below a certain threshold also 0: can parameterize this eventually
  x = np.copy(similarity_matrix)
  np.fill_diagonal(x, 0)
  x[x < threshold] = 0
  G = nx.from_numpy_matrix(np.matrix(x), create_using=nx.DiGraph)
  return G

def random_walk(word, n_steps):
  nodes_visited = []
  random_node = list(vocab.vocab_word).index(word) 
  nodes_visited.append(random_node)
  dict_counter = {} #initialise the value for all nodes as 0 (i.e., each node has been visited 0 times)
  for i in range(Graph.number_of_nodes()):
      dict_counter[i] = 0
  # update dict_count for the chosen random node by 1 (since walk starts here)
  dict_counter[random_node] = dict_counter[random_node]+1

  #Traversing through the neighbors of start node
  #increment by traversing through all neighbors nodes ()
  for i in range(n_steps):
      list_for_nodes = list(Graph.neighbors(random_node))
      neighbors = {}
      for n in list_for_nodes:
        neighbors[n] = Graph.edges[(random_node,n)]['weight']
      if len(list_for_nodes)==0:
        # if random_node having no outgoing edges
        # choose a different random node and update its visit count
        random_node = random.choice([i for i in range(Graph.number_of_nodes())])
        dict_counter[random_node] = dict_counter[random_node]+1
        nodes_visited.append(random_node)
      else:
        #choose a node randomly from neighbors of current random_node
        random_node = random.choices(population = list_for_nodes, k = 1,
                weights=list(neighbors.values()))[0]
        dict_counter[random_node] = dict_counter[random_node]+1
        nodes_visited.append(random_node)
          
  ## return the words visited and the counts of each time a node was visited (in the order of vocab) as a numpy array

  words_visited = [list(vocab.vocab_word)[index] for index in nodes_visited]

  return words_visited, np.fromiter(dict_counter.values(), dtype=float)

def union_intersection(w1,w2, n_steps, n_walks, vocabulary):
  """ starts n_walks independent random walks for n_steps, and computes union and intersection """
  rw_w1 = np.sum(np.array([random_walk(w1, n_steps)[1] for i in range(n_walks)]), axis = 0)
  rw_w2 = np.sum(np.array([random_walk(w2, n_steps)[1] for i in range(n_walks)]), axis = 0)

  v = vocabulary.copy()

  v["w1_visited_count"] = rw_w1.tolist()
  v["w2_visited_count"] = rw_w2.tolist()

  v["w1*w2"] = v["w1_visited_count"]*v["w2_visited_count"]

  ## compute non-zero, i.e., all visited nodes 

  nonzero_w1 = list(v.loc[v['w1_visited_count'] != 0].vocab_word)
  nonzero_w2 = list(v.loc[v['w2_visited_count'] != 0].vocab_word)

  ## compute union and intersection

  union = set(nonzero_w1).union(set(nonzero_w2))
  intersection = set(nonzero_w1).intersection(set(nonzero_w2))

  ## we also need the counts of these visited nodes, sorted by nodes visited highly by both words

  union_df = v.loc[v['vocab_word'].isin(list(union))].sort_values(by='w1*w2', ascending=False)
  intersection_df = v.loc[v['vocab_word'].isin(list(intersection))].sort_values(by='w1*w2', ascending=False)

  return union_df, intersection_df

def predication_vector(sim_matrix, w1, w2, m, k):
  """
  # computes a predication vector based on kintsch's predication algorithm
  # in this algorithm, first m neighbors of w1 are computed
  # a network is created using m, w1, and w2 with inhibitory links between the m nodes and all cosines b/w m and w1 and w2
  # this network is then "integrated" until steady state, and then
  # a centroid is calculated using w1, w2, and k strongest neighbors of w2
  ## details of spreading activation:
  # More specific, an activation vector representing the initial activation
  # values of all nodes in the net is postmultiplied repeatedly with
  # the connectivity matrix. After each multiplication the activation
  # values are renormalized: Negative values are set to zero,
  # and each of the positive activation values is divided by the sum
  # of all activation values, so that the total activation on each cycles
  # remains at a value of one (e.g., Rumelhart & McClelland,
  # 1986). Usually, the system finds a stable state fairly rapidly;
  """
  neighbors_of_w1 = sim_matrix[list(vocab.vocab_word).index(w1)]
  topn_indices = np.argpartition(neighbors_of_w1, -m)[-m:].tolist()
  nodes_indices = list(set(topn_indices + [list(vocab.vocab_word).index(w1),list(vocab.vocab_word).index(w2) ]))
  nodes = [list(vocab.vocab_word)[i] for i in nodes_indices]
  vecs = representations['glove'][nodes_indices]
  # construct nodes similarity matrix 
  s = create_similarity_matrix(vecs, vecs, vecs.shape[0])
  w1_index = nodes.index(w1)
  w2_index = nodes.index(w2)
  valid_indices = np.arange(0,s.shape[1]).tolist()
  valid_indices.pop(w1_index)
  valid_indices.pop(w2_index)
  s[:, valid_indices] = 0
  # also set connections to self = 0
  s[w1_index, w1_index] = 0
  s[w2_index, w2_index] = 0
  n_zero = s.size - np.count_nonzero(s)
  sum_positive_activations = np.sum(s)
  # set all 0 values to sum/n_zero
  s[s == 0] = -sum_positive_activations/n_zero

  ## now we perform the spreading activation integration: restricted to 5 steps
  s_integrated = kintsch_integration(s, 5)
  ## now we extract k neighbors of w2

  neighbors_of_w2 = s_integrated[w2_index]
  topn_indices = np.argpartition(neighbors_of_w2, -k)[-k:].tolist()
  w2_words = [nodes[i] for i in topn_indices]
  indices_in_vocab = [list(vocab.vocab_word).index(i) for i in w2_words]
  nodes_indices = list(set(indices_in_vocab + [list(vocab.vocab_word).index(w1),list(vocab.vocab_word).index(w2) ]))
  finalnodes = [list(vocab.vocab_word)[i] for i in nodes_indices]
  vecs = representations['glove'][nodes_indices]
  ## next we compute centroid of w1, w2, and w2_words
  centroid = np.mean(vecs, axis = 0).reshape((1,vecs.shape[1]))
  close = find_closest(centroid, vocab, representations['glove'])
  return  close
  
def kintsch_integration(m, n_times):
  for i in range(1,n_times):
    m = m**(i+1)
    m[m <0] = 0
    sum_positive_activations = np.sum(m)
    m = m/sum_positive_activations
  return m

def find_closest(vector, vocab, embeddings, k = 10):
  """finds the words closest to given vector in a given vocab for given embeddings"""
  cosine = 1-scipy.spatial.distance.cdist(embeddings, vector, 'cosine')
  cosine = cosine.flatten()
  centroid_indices = np.argpartition(cosine, -k)[-k:].tolist()
  centroid_words = [list(vocab.vocab_word)[i] for i in centroid_indices]
  return centroid_words    

In [226]:
predication_vector(sim_matrix_glove, "happy", "sad", 200, 5)

['feel',
 'always',
 'love',
 'really',
 'sad',
 'thing',
 'happy',
 'everyone',
 'something',
 'want']

# Speaker functions
These are directly copied over from the cogsci notebook except for the last two functions that restrict the RSA functions to set of candidates passed as function arguments

In [6]:
parentfolder = "/content/drive/My Drive/Connector-CogSci2021/Descriptive analyses and model predictions/"
with open(parentfolder + 'boards.json', 'r') as json_file:
    boards = json.load(json_file)

expdata = pd.read_csv(parentfolder + "final_board_clues_all.csv", encoding= 'unicode_escape')
sample_df = pd.read_csv(parentfolder + "vocab.csv")

def compute_board_combos(board_name):
    board = boards[board_name]
    all_possible_combs = list(itertools.combinations(board, 2))
    combs_df = pd.DataFrame(all_possible_combs, columns =['Word1', 'Word2'])
    combs_df["wordpair"] = combs_df["Word1"] + '-'+ combs_df["Word2"]
    return combs_df

allcombinations_df = pd.DataFrame(columns=['Board', 'Word1','Word2', 'wordpair'])
board_combos = {board_name : compute_board_combos(board_name) for board_name in boards.keys()}

for board in board_combos:
    newdf = board_combos[board]
    newdf.insert(loc=0, column='Board', value=board)
    allcombinations_df = pd.concat([allcombinations_df, newdf])

def get_wordpair_list(board_combos, board_name) :
    return list(board_combos[board_name]['wordpair'])

def create_board_matrix(combs_df, context_board, embeddings):
    # grab subset of words in given board and their corresponding glove vectors
    board_df = sample_df[sample_df['Word'].isin(context_board)]
    board_word_indices = list(board_df.index)
    board_words = board_df["Word"]
    board_vectors = embeddings[board_word_indices]

    ## clue_sims is the similarity of ALL clues in full searchspace (size N) to EACH word on board (size 20)
    clue_sims = 1 - scipy.spatial.distance.cdist(board_vectors, embeddings, 'cosine')

    ## once we have the similarities of the clue to the words on the board
    ## we define a multiplicative function that maximizes these similarities
    board_df.reset_index(inplace = True)

    ## next we find the product of similarities between c-w1 and c-w2 for that specific board's 190 word-pairs
    ## this gives us a 190 x N array of product similarities for a given combs_df
    ## specifically, for each possible pair, pull out 
    f_w1_list =  np.array([clue_sims[board_df[board_df["Word"]==row["Word1"]].index.values[0]]
                         for  index, row in combs_df.iterrows()])
    f_w2_list =  np.array([clue_sims[board_df[board_df["Word"]==row["Word2"]].index.values[0]] 
                         for  index, row in combs_df.iterrows()])

    # result is of length 190 for the product of similarities (i.e. how similar each word i is to BOTH in pair)
    # note that cosine is in range [-1, 1] so we have to convert to [0,1] for this conjunction to be valid
    return ((f_w1_list + 1) /2) * ((f_w2_list + 1)/2)

board_matrices = {
    key : {board_name : create_board_matrix(board_combos[board_name], boards[board_name], embedding) 
           for board_name in boards.keys()}
    for (key, embedding) in representations.items()
}

def literal_guesser_np(board_name, representation):
    boardmatrix = board_matrices[representation][board_name]
    return softmax(boardmatrix, axis=0)

def pragmatic_speaker_np(board_name, beta, costweight, representation):
    literal_guesser_prob = np.log(literal_guesser_np(board_name, representation))
    clues_cost = -np.array(list(sample_df["LgSUBTLWF"]))
    utility = (1-costweight) * literal_guesser_prob - costweight * clues_cost
    return softmax(beta * utility, axis = 1)

## define new "candidate" functions

def pragmatic_speaker_candidates(board_name, beta, costweight, representation, candidates):
    candidate_index = [list(sample_df["Word"]).index(w) for w in candidates]
    literal_guesser_prob = np.log(literal_guesser_candidates(board_name, representation, candidates))
    clues_cost = -np.array([list(sample_df["LgSUBTLWF"])[i] for i in candidate_index])
    utility = (1-costweight) * literal_guesser_prob - costweight * clues_cost
    return softmax(beta * utility, axis = 1)

def literal_guesser_candidates(board_name, representation, candidates):
    boardmatrix = board_matrices[representation][board_name]
    ## here we restrict the softmax to only certain candidates
    candidate_index = [list(sample_df["Word"]).index(w) for w in candidates]
    restricted_boardmatrix = boardmatrix[:,candidate_index]
    return softmax(restricted_boardmatrix, axis=0)


## Running an example

In [8]:
threshold = 0.4
sim_matrix_glove = create_similarity_matrix(representations['glove'], representations['glove'], representations['glove'].shape[0])
Graph = create_graph(sim_matrix_glove, threshold)

In [9]:
target = 'trauma-weird'
w1, w2 = target.split(sep = "-")
n_steps = 5
n_walks = 1000

print(f"example of a random walk of {n_steps} steps from {w1}",random_walk(w1, n_steps))
print(f"example of a random walk of {n_steps} steps from {w2}",random_walk(w2, n_steps))

# computing union and intersection of independent walks
u, i = union_intersection(w1,w2, n_steps, n_walks, vocab)

union_candidates = list(u.vocab_word)
int_candidates = list(i.vocab_word)

print(f"{len(u)} items in union: {union_candidates[:5]}")
print(f"{len(i)} items in intersection: {int_candidates[:5]}")

example of a random walk of 5 steps from trauma (['trauma', 'mental', 'criminal', 'could', 'age', 'disability'], array([0., 0., 0., ..., 0., 0., 0.]))
example of a random walk of 5 steps from weird (['weird', 'sexy', 'dirty', 'boring', 'depressing', 'awful'], array([0., 0., 0., ..., 0., 0., 0.]))
3067 items in union: ['horrific', 'experience', 'kind', 'sort', 'something']
689 items in intersection: ['horrific', 'experience', 'something', 'kind', 'sort']


In [ ]:
clue = 'study'
n = 5
wordpairlist = get_wordpair_list(board_combos, 'e1_board1_words')
target_index = wordpairlist.index(target)
clue_index = list(sample_df["Word"]).index(clue)

a = literal_guesser_np('e1_board1_words', 'glove')[:,clue_index]
y = pragmatic_speaker_np('e1_board1_words', 18.858, 0.004, 'glove')

b_union = literal_guesser_candidates('e1_board1_words', 'glove', union_candidates)[:, union_candidates.index(clue)]
c_union = pragmatic_speaker_candidates('e1_board1_words', 18.858, 0.004, 'glove', union_candidates)

b_int = literal_guesser_candidates('e1_board1_words', 'glove', int_candidates)[:, int_candidates.index(clue)]
c_int = pragmatic_speaker_candidates('e1_board1_words', 18.858, 0.004, 'glove', int_candidates)


print(f"for wordpair {target} and clue {clue}")

print(f"considering literal guesser predictions....")

print("literal guesser prediction is:", wordpairlist[np.argmax(a)])
print("literal candidate UNION prediction is:", wordpairlist[np.argmax(b_union)])
print("literal candidate INTERSECTION prediction is:", wordpairlist[np.argmax(b_int)])

print(f"considering pragmatic guesser predictions....")

top = y[target_index,:].argsort()[-5:][::-1].tolist()
top_words = [list(sample_df["Word"])[x] for x in top]
print(f"top {n} prag speaker predictions are:", top_words)

top = c_union[target_index,:].argsort()[-5:][::-1].tolist()
top_words = [list(u.vocab_word)[x] for x in top]
print(f"top {n} prag candidate UNION are:", top_words)

top = c_int[target_index,:].argsort()[-5:][::-1].tolist()
top_words = [list(i.vocab_word)[x] for x in top]
print(f"top {n} prag candidate INTERSECTION are:", top_words)

print(f"considering top {n} nodes visited by union and intersection...")
print(f"highly visited nodes in the union: {union_candidates[:n]}")
print(f"highly visited nodes in the intersection: {int_candidates[:n]}")


for wordpair exam-algebra and clue study
considering literal guesser predictions....
literal guesser prediction is: write-better
literal candidate UNION prediction is: write-better
literal candidate INTERSECTION prediction is: write-better
considering pragmatic guesser predictions....
top 5 prag speaker predictions are: ['algebra', 'exam', 'mathematics', 'exams', 'calculus']
top 5 prag candidate UNION are: ['algebra', 'exam', 'mathematics', 'exams', 'calculus']
top 5 prag candidate INTERSECTION are: ['algebra', 'mathematics', 'math', 'grade', 'geometry']
considering top 5 nodes visited by union and intersection...
highly visited nodes in the union: ['mathematics', 'algebra', 'mathematical', 'arithmetic', 'finite']
highly visited nodes in the intersection: ['mathematics', 'algebra', 'mathematical', 'arithmetic', 'finite']
